# Select Your IPTS 

In [ ]:
# #from __code.filename_metadata_match import FilenameMetadataMatch

# from __code import system
# system.System.select_working_dir()
# from __code.__all import custom_style
# custom_style.style()

In [ ]:
import ipywe.fileselector
from IPython.core.display import HTML
from __code.time_utility import RetrieveTimeStamp


class FilenameMetadataMatch(object):

    data_folder = ''
    metadata_file = ''
    
    list_data_time_stamp = None
    
    def __init__(self, working_dir='./'):
        self.working_dir = working_dir

    def select_input_folder(self):
        _instruction = "Select Input Folder ..."
        self.input_folder_ui = ipywe.fileselector.FileSelectorPanel(instruction=_instruction,
                                                                    start_dir=self.working_dir,
                                                                    next=self.select_input_folder_done,
                                                                    type='directory',
                                                                   )
        self.input_folder_ui.show()
        
    def select_input_folder_done(self, folder):
        self.data_folder = folder
        display(HTML('Folder Selected: <span style="font-size: 20px; color:green">' + folder))

    def select_metadata_file(self):
        _instruction = "Select Metadata File ..."
        self.metadata_ui = ipywe.fileselector.FileSelectorPanel(instruction=_instruction,
                                                                start_dir=self.working_dir,
                                                                next=self.select_metadata_file_done,
                                                               )
        self.metadata_ui.show()
        
    def select_metadata_file_done(self, metadata_file):
        self.metadata_file = metadata_file
        display(HTML('Metadata File Selected: <span style="font-size: 20px; color:green">' + metadata_file))
        
    def retrieve_time_stamp(self):
        o_retriever = RetrieveTimeStamp(folder=self.data_folder)
        o_retriever._run()
        self.list_data_time_stamp = o_retriever
    
    def load_metadata(self):
        metadata_file = self.metadata_file
    

# Select Input Folder 

In [ ]:
#o_match = FilenameMetadataMatch(working_dir=system.System.get_working_dir())
o_match = FilenameMetadataMatch(working_dir='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-20444-Regina/')
o_match.select_input_folder()

# Select Metadata File 

In [ ]:
o_match.select_metadata_file()

# Retrieve Time Stamp

In [ ]:
o_match.retrieve_time_stamp()

# Load Metadata File

# DEBUGGING STARTS HERE

In [ ]:
import pandas as pd
import codecs
from __code.file_handler import get_file_extension
from ipywidgets import widgets

In [ ]:
#metadata_file = o_match.metadata_file
metadata_file = '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-20444-Regina/metadata_files/A49_3_at80C_0_01_OCV_C03.mpt'
assert os.path.exists(metadata_file)

In [ ]:
class MPTFileParser(object):
    
    nbr_row_metadata = 47
    
    metadata_dict = {'Acquisition started on': {"split1": ' : ',
                                                "value": '',
                                                "split2": '',
                                               },
                     'Electrode surface area': {"split1": ' : ',
                                                "value": '',
                                                "split2": ' ',
                                                "units": '',
                                               },                     
                    }
        
    time_info_index_column = 2 # in this file, the time information is in the column #2

    def __init__(self, filename=''):
        self.filename = filename
        self.parse()
    
    def parse(self):
        self.read_data()
        self.read_metadata()
        
    def read_data(self):
        o_pd = pd.read_csv(self.filename,
                           sep='\t',
                          skiprows=47,
                          )
        self.o_pd = o_pd
        
    def read_metadata(self):
        fdata = codecs.open(self.filename,
                           'r',
                           encoding='utf_8',
                           errors='ignore')
        metadata = []
        for _row in np.arange(self.nbr_row_metadata):
            metadata.append(fdata.readline())
        self.metadata = metadata

        self.read_specific_metadata()
        
    def read_specific_metadata(self):
        for _keys in self.metadata_dict.keys():
            for _line in self.metadata:
                if _keys in _line:
                    result = _line.split(self.metadata_dict[_keys]['split1']) #1st split
                    if not self.metadata_dict[_keys]['split2']:
                        self.metadata_dict[_keys]['value'] = result[1].strip()
                    else: #2nd split
                        [value, units] = result[1].strip().split(self.metadata_dict[_keys]['split2'])
                        self.metadata_dict[_keys]['value'] = value.strip()
                        self.metadata_dict[_keys]['units'] = units.strip()

                
class MetadataFileParser(object):
    
    filename = ''
    meta_type = ''
    
    def __init__(self, filename='', meta_type=''):
        self.filename = filename
        if meta_type:
            self.meta_type = meta_type
        else:
            self.meta_type = get_file_extension(filename)
                        
    def parse(self):
        if self.meta_type == 'mpt':
            o_mpt = MPTFileParser(filename=self.filename)
            self.meta = o_mpt
        else:
            raise NotImplementedError("This file format is not supported!")
        
    def create_metadata_selected_vs_time_stamp(self, metadata_selected):
        #time_column = self.meta.o_pd.
        pass
        

        
        

In [ ]:
o_meta = MetadataFileParser(filename=metadata_file)
o_meta.parse()

In [ ]:
o_meta.meta.metadata_dict

In [ ]:
# get list of columns

In [ ]:
list_columns = list(o_meta.meta.o_pd.columns.values)
list_columns.pop(o_meta.meta.time_info_index_column)
print(list_columns)

In [ ]:
box = widgets.HBox([widgets.Label("Select Metadata:",
                                layout=widgets.Layout(width='15%'),
                                ),
                   widgets.Dropdown(options=list_columns,
                                   value=list_columns[0],
                                   layout=widgets.Layout(width='30%')),
                   ])
display(box)
                   

In [ ]:
metadata_selected = box.children[1].value
metadata_selected